In [39]:
import openai

from dotenv import load_dotenv, find_dotenv

import pandas as pd
pd.set_option('display.max_colwidth', 0)

dot_env_file_exist = load_dotenv(find_dotenv()) # read local .env file
# Returns true or false if .env exists in current directory
print('dot_env_file_exist: ', dot_env_file_exist)


dot_env_file_exist:  True


In [40]:
# PYTHON Solution: Read the key directly from the .env file

def python_get_dotenv_data(parms):
    with open('.env', 'r') as reader:
        out = reader.readlines()
    for i in parms:
        for j in range(len(out)):
            ind = out[j].rfind(i)
            if ind != -1:
                st = ind+(2+len(i))
                end = len(out[j]) - 2
                globals()[f'{i}'] = out[j][st:end]

# Save key as a Python variable
parms = ['OPENAI_API_KEY']
python_get_dotenv_data(parms)
openai.api_key = OPENAI_API_KEY
print(openai.api_key)

sk-XlzTikk7bPnWVOBFDeTFT3BlbkFJaf3m4VuJu9AfjyDVKYm


In [ ]:
from dotenv import load_dotenv
import os

# Lade die Umgebungsvariablen aus der .env-Datei
load_dotenv()

# Speichere den OpenAI API-Key als Variable
openai_api_key = os.getenv('OPENAI_API_KEY')

# Verwende den API-Key nach Bedarf
print(openai_api_key)  # Nur zur Demonstration, entferne dies in der Produktionsumgebung


In [41]:
# Define the client
from openai import OpenAI
import openai
client = openai.api_key

In [42]:
# Starten des Upload-Prozess Training-Daten
import openai
client = OpenAI()

my_training_data = client.files.create(
    file=open("data/training_set.jsonl", "rb"),
    purpose='fine-tune'
)

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-AIpV9***************************************SGpB. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [17]:
print(my_training_data.id)

file-yJYIO2sHWmrGWGYhKoJTUGI9


In [19]:
# Starten des Upload-Prozess Validierung-Daten
import openai
client = OpenAI()

my_eval_data = client.files.create(
    file=open("data/validation_set.jsonl", "rb"),
    purpose='fine-tune'
)

In [20]:
print(my_eval_data.id)

file-FEfZVH9BPHodBaQLXQsYTo0Z


In [21]:
# Start das Finetuning
from openai import OpenAI
client = OpenAI()

response= client.fine_tuning.jobs.create(
  training_file=my_training_data.id,
  validation_file=my_eval_data.id,
  model="gpt-3.5-turbo"
)

response

FineTuningJob(id='ftjob-ZdaAd5mj3iHHbZ9FXiUVoNo4', created_at=1711979551, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-4iR8mcaw4LEVCNvoGGcPHEEv', result_files=[], status='validating_files', trained_tokens=None, training_file='file-yJYIO2sHWmrGWGYhKoJTUGI9', validation_file='file-FEfZVH9BPHodBaQLXQsYTo0Z', user_provided_suffix=None)

In [22]:
job_id = response.id
job_id

'ftjob-ZdaAd5mj3iHHbZ9FXiUVoNo4'

In [26]:
# Zeige den Status des Fine-Tuning an
the_job = client.fine_tuning.jobs.retrieve(job_id)
the_job

FineTuningJob(id='ftjob-ZdaAd5mj3iHHbZ9FXiUVoNo4', created_at=1711979551, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:christian-kolb::99CSQB6v', finished_at=1711979865, hyperparameters=Hyperparameters(n_epochs=10, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-4iR8mcaw4LEVCNvoGGcPHEEv', result_files=['file-dUFLE9R4x9iY9PdGSrLOW1TI'], status='succeeded', trained_tokens=5470, training_file='file-yJYIO2sHWmrGWGYhKoJTUGI9', validation_file='file-FEfZVH9BPHodBaQLXQsYTo0Z', user_provided_suffix=None)

In [27]:
# Zeig die Job ID
print("Job ID: ", the_job.id)
#Zeig den Status des Jobs
print("Job Status: ", the_job.status)
# Zeig die Anzahl der trainierten Tokens an
print("Die trainierten Tokens: ", the_job.trained_tokens)
# Zeige den Namen von dem neuen Fintuned Modell an
print("Name des neuen Fintuned Modells: ", the_job.fine_tuned_model)

Job ID:  ftjob-ZdaAd5mj3iHHbZ9FXiUVoNo4
Job Status:  succeeded
Die trainierten Tokens:  5470
Name des neuen Fintuned Modells:  ft:gpt-3.5-turbo-0125:christian-kolb::99CSQB6v


Nützliche Codes

In [ ]:
#Stoppe einen Job
client.fine_tuning.jobs.cancel ("ftjob-abc1223")

#Lösche eine finetuned Modell
client.models.delete ("ftmodel-abc1223")

Teste das neue Model

In [30]:
def test_new_model(prompt):
    #Stelle einer Antwort von dem Modell
    response = client.chat.completions.create(
        model="ft:gpt-3.5-turbo-0125:christian-kolb::99CSQB6v",
        messages= [
            {
                "role": "system", "content": "Clippy, the witty chatbot, blending facts with a touch of sarcasm.",
                "role": "user", "content": prompt
                }
        ]
    )
    #Gibt die Antwort aus
    print(response.choices[0].message.content)

In [31]:
test_new_model("Can you tell me about the theory of relativity?")

The theory of relativity is a fundamental theory in physics that describes the relationship between space and time, and their interactions with matter and energy. It was developed by Albert Einstein in the early 20th century and has had a profound impact on our understanding of the universe.

There are two main aspects of the theory of relativity: special relativity and general relativity. 

Special relativity is concerned with the behavior of objects moving at speeds close to the speed of light. It introduces the concept that the laws of physics are the same for all observers, regardless of their relative motion. Special relativity also leads to the famous equation E=mc^2, which describes the equivalence of mass and energy.

General relativity, on the other hand, extends the principles of special relativity to include gravity. It posits that massive objects warp the fabric of spacetime, causing objects to move along curved paths. General relativity has been successful in explaining ph

In [ ]:
#Cancel a Job
client.fine_tuning.jobs.cancel("job_123")

#Lösche a Fine.tuned Model
client.models.delete("ft:model_name")